In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

zip_path = "/content/drive/MyDrive/image_align_celeba.zip"  # ⬅️ change path here
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/img_align_celeba")

print("✅ Unzipped to /content/img_align_celeba/")


✅ Unzipped to /content/img_align_celeba/


In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models


In [4]:
# Load CSVs
skin_df = pd.read_csv("/content/drive/MyDrive/celeba_labeled_skin_full.csv")
shade_df = pd.read_csv("/content/drive/MyDrive/foundation_shade_mapping_expanded.csv")

# Merge to get recommended shades
merged_df = pd.merge(skin_df, shade_df, on=["SkinTone", "Undertone"], how="left")
merged_df.dropna(subset=["RecommendedShades"], inplace=True)

# Label Encoding
skin_enc = LabelEncoder()
under_enc = LabelEncoder()
shade_enc = LabelEncoder()

merged_df["SkinToneClass"] = skin_enc.fit_transform(merged_df["SkinTone"])
merged_df["UndertoneClass"] = under_enc.fit_transform(merged_df["Undertone"])
merged_df["ShadeClass"] = shade_enc.fit_transform(merged_df["RecommendedShades"])

# Split
train_df, val_df = train_test_split(merged_df, test_size=0.1, random_state=42)

# Image directory
image_dir = "/content/img_align_celeba/img_align_celeba/img_align_celeba"


In [5]:
class CelebaShadeDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row["Image"])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        skin_label = row["SkinToneClass"]
        under_label = row["UndertoneClass"]
        shade_label = row["ShadeClass"]
        return image, skin_label, under_label, shade_label


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = CelebaShadeDataset(train_df, image_dir, transform)
val_dataset = CelebaShadeDataset(val_df, image_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)


In [7]:
import torch
import torch.nn as nn
from torchvision import models

class ShadePredictorCNN(nn.Module):
    def __init__(self, num_skin, num_under, num_shade):
        super(ShadePredictorCNN, self).__init__()

        resnet = models.resnet18(pretrained=True)
        self.base = nn.Sequential(*list(resnet.children())[:-1])  # remove the FC layer

        self.head_skin = nn.Linear(512, num_skin)
        self.head_under = nn.Linear(512, num_under)
        self.head_shade = nn.Linear(512, num_shade)

    def forward(self, x):
        x = self.base(x)              # output shape: (batch_size, 512, 1, 1)
        x = torch.flatten(x, 1)       # shape: (batch_size, 512)

        out_skin = self.head_skin(x)
        out_under = self.head_under(x)
        out_shade = self.head_shade(x)

        return out_skin, out_under, out_shade


In [8]:
model = ShadePredictorCNN(
    num_skin=len(skin_enc.classes_),
    num_under=len(under_enc.classes_),
    num_shade=len(shade_enc.classes_)
)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 130MB/s]


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion_skin = nn.CrossEntropyLoss()
criterion_under = nn.CrossEntropyLoss()
criterion_shade = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [10]:
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_skin = correct_under = correct_shade = 0

    for images, skin_labels, under_labels, shade_labels in tqdm(train_loader):
        images = images.to(device)
        skin_labels = skin_labels.to(device)
        under_labels = under_labels.to(device)
        shade_labels = shade_labels.to(device)

        out_skin, out_under, out_shade = model(images)

        loss_skin = criterion_skin(out_skin, skin_labels)
        loss_under = criterion_under(out_under, under_labels)
        loss_shade = criterion_shade(out_shade, shade_labels)
        loss = loss_skin + loss_under + loss_shade

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct_skin += (out_skin.argmax(1) == skin_labels).sum().item()
        correct_under += (out_under.argmax(1) == under_labels).sum().item()
        correct_shade += (out_shade.argmax(1) == shade_labels).sum().item()

    total = len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f} | "
          f"Skin Acc: {correct_skin/total:.4f} | "
          f"Undertone Acc: {correct_under/total:.4f} | "
          f"Shade Acc: {correct_shade/total:.4f}")


100%|██████████| 2839/2839 [10:29<00:00,  4.51it/s]


Epoch 1/10 | Loss: 3784.2523 | Skin Acc: 0.8141 | Undertone Acc: 0.9049 | Shade Acc: 0.7406


100%|██████████| 2839/2839 [10:22<00:00,  4.56it/s]


Epoch 2/10 | Loss: 2889.7344 | Skin Acc: 0.8543 | Undertone Acc: 0.9302 | Shade Acc: 0.7962


100%|██████████| 2839/2839 [10:28<00:00,  4.52it/s]


...

Epoch 8/10 | Loss: 989.3372 | Skin Acc: 0.9536 | Undertone Acc: 0.9792 | Shade Acc: 0.9350


100%|██████████| 2839/2839 [10:25<00:00,  4.54it/s]


Epoch 9/10 | Loss: 776.0266 | Skin Acc: 0.9644 | Undertone Acc: 0.9838 | Shade Acc: 0.9495


100%|██████████| 2839/2839 [10:27<00:00,  4.53it/s]

Epoch 10/10 | Loss: 632.3020 | Skin Acc: 0.9716 | Undertone Acc: 0.9864 | Shade Acc: 0.9590


In [11]:
model.eval()
correct_skin = correct_under = correct_shade = total = 0

with torch.no_grad():
    for x, y_skin, y_under, y_shade in val_loader:
        x, y_skin, y_under, y_shade = x.to(device), y_skin.to(device), y_under.to(device), y_shade.to(device)
        out_skin, out_under, out_shade = model(x)

        pred_skin = torch.argmax(out_skin, dim=1)
        pred_under = torch.argmax(out_under, dim=1)
        pred_shade = torch.argmax(out_shade, dim=1)

        correct_skin += (pred_skin == y_skin).sum().item()
        correct_under += (pred_under == y_under).sum().item()
        correct_shade += (pred_shade == y_shade).sum().item()
        total += y_skin.size(0)

print(f"✅ Final Accuracy | Skin Tone: {correct_skin/total:.4f} | Undertone: {correct_under/total:.4f} | Foundation Shade: {correct_shade/total:.4f}")


✅ Final Accuracy | Skin Tone: 0.8720 | Undertone: 0.9141 | Foundation Shade: 0.7970


In [12]:
import joblib

# Save the model state
torch.save(model.state_dict(), "shade_predictor_model.pth")

# Save the label encoders
joblib.dump(skin_enc, "skin_encoder.pkl")
joblib.dump(under_enc, "under_encoder.pkl")
joblib.dump(shade_enc, "shade_encoder.pkl")

print("✅ Model and encoders saved.")


✅ Model and encoders saved.


In [13]:
import joblib
from torchvision import models

# Load encoders
skin_enc = joblib.load("skin_encoder.pkl")
under_enc = joblib.load("under_encoder.pkl")
shade_enc = joblib.load("shade_encoder.pkl")

# Re-initialize the model with correct output sizes
model = ShadePredictorCNN(
    num_skin=len(skin_enc.classes_),
    num_under=len(under_enc.classes_),
    num_shade=len(shade_enc.classes_)
)

# Load trained weights
model.load_state_dict(torch.load("shade_predictor_model.pth", map_location=device))
model.to(device)
model.eval()

print("✅ Model and encoders loaded.")


✅ Model and encoders loaded.


In [14]:
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ResNet ImageNet mean
        std=[0.229, 0.224, 0.225]    # ResNet ImageNet std
    )
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)


In [15]:
def predict_image(image_path):
    img_tensor = preprocess_image(image_path)

    with torch.no_grad():
        out_skin, out_under, out_shade = model(img_tensor)

    # Get predicted class indices
    skin_idx = torch.argmax(out_skin, dim=1).item()
    under_idx = torch.argmax(out_under, dim=1).item()
    shade_idx = torch.argmax(out_shade, dim=1).item()

    # Decode class labels
    skin_label = skin_enc.inverse_transform([skin_idx])[0]
    under_label = under_enc.inverse_transform([under_idx])[0]
    shade_label = shade_enc.inverse_transform([shade_idx])[0]

    return {
        "Skin Tone": skin_label,
        "Undertone": under_label,
        "Shade": shade_label
    }


In [18]:
import joblib
import torch

# Save the model
torch.save(model.state_dict(), "shade_predictor_model.pth")

# Save the encoders
joblib.dump(skin_enc, "skin_enc.pkl")
joblib.dump(under_enc, "under_enc.pkl")
joblib.dump(shade_enc, "shade_enc.pkl")

print("✅ Model and encoders saved!")


✅ Model and encoders saved!


In [21]:
import gradio as gr
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import joblib

# Load encoders
skin_enc = joblib.load("skin_enc.pkl")
under_enc = joblib.load("under_enc.pkl")
shade_enc = joblib.load("shade_enc.pkl")

# Correct class sizes based on encoders
num_skin = len(skin_enc.classes_)
num_under = len(under_enc.classes_)
num_shade = len(shade_enc.classes_)

# Define the model (multi-head version)
class ShadePredictorCNN(nn.Module):
    def __init__(self, num_skin, num_under, num_shade):
        super(ShadePredictorCNN, self).__init__()
        resnet = models.resnet18(weights=None)
        self.base = nn.Sequential(*list(resnet.children())[:-1])

        self.head_skin = nn.Linear(512, num_skin)
        self.head_under = nn.Linear(512, num_under)
        self.head_shade = nn.Linear(512, num_shade)

    def forward(self, x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        return self.head_skin(x), self.head_under(x), self.head_shade(x)

# Load model and weights
model = ShadePredictorCNN(num_skin=num_skin, num_under=num_under, num_shade=num_shade)
model.load_state_dict(torch.load("shade_predictor_model.pth", map_location="cpu"))
model.eval()

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Prediction function
def predict(image: Image.Image):
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        _, _, shade_logits = model(image)
        pred_idx = shade_logits.argmax(dim=1).item()
        shade_label = shade_enc.inverse_transform([pred_idx])[0]
    return f"Recommended Foundation Shade: {shade_label}"

# Launch Gradio interface
gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Facial Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="TrueTone Foundation Shade Predictor",
    description="Upload a facial image to get a recommended foundation shade."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c10f2dd8c8270c5dbc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
